# Pubmed Metadata ETL

In this exercise, I will demonstrate extracting article metadata from PubMed articles. Pubmed articles are publically available as XMLs here: https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_bulk/. There are daily updates and this page should contain all articles.

In [1]:
## required imports
import requests
import bs4
import gzip
import shutil
from io import BytesIO
from bs4 import BeautifulSoup
import tarfile
import os
import pandas as pd
from tqdm import tqdm
import re
from functools import reduce

I will use the requests library to pull data directly from the site. I'll go into the oa_comm folder (commercially available articles) and into the xml folder.

In [2]:
url = "https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_bulk/oa_comm/xml/"
r = requests.get(url) ## pull data from specified url
r.text[0:2000]

'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">\n<html>\n <head>\n  <title>Index of /pub/pmc/oa_bulk/oa_comm/xml</title>\n </head>\n <body>\n<h1>Index of /pub/pmc/oa_bulk/oa_comm/xml</h1>\n<pre>Name                                                      Last modified      Size  <hr><a href="/pub/pmc/oa_bulk/oa_comm/">Parent Directory</a>                                                               -   \n<a href="oa_comm_xml.PMC000xxxxxx.baseline.2023-12-18.filelist.csv">oa_comm_xml.PMC000xxxxxx.baseline.2023-12-18.filelist.csv</a> 2023-12-18 01:15  335K  \n<a href="oa_comm_xml.PMC000xxxxxx.baseline.2023-12-18.filelist.txt">oa_comm_xml.PMC000xxxxxx.baseline.2023-12-18.filelist.txt</a> 2023-12-18 01:15  318K  \n<a href="oa_comm_xml.PMC000xxxxxx.baseline.2023-12-18.tar.gz">oa_comm_xml.PMC000xxxxxx.baseline.2023-12-18.tar.gz</a>       2023-12-18 01:16   42M  \n<a href="oa_comm_xml.PMC001xxxxxx.baseline.2023-12-18.filelist.csv">oa_comm_xml.PMC001xxxxxx.baseline.2023-12-18.filelist.c

The request returns the oa_comm/xml url's html file, and you can see that the files under this path are listed with html tag 'a'. In order to extract these file names, I will use the BeautifulSoup library.

In [3]:
data = bs4.BeautifulSoup(r.text, "html.parser")
files = data.find_all('a') ## get all instances with tag 'a'
files[0:7]

[<a href="/pub/pmc/oa_bulk/oa_comm/">Parent Directory</a>,
 <a href="oa_comm_xml.PMC000xxxxxx.baseline.2023-12-18.filelist.csv">oa_comm_xml.PMC000xxxxxx.baseline.2023-12-18.filelist.csv</a>,
 <a href="oa_comm_xml.PMC000xxxxxx.baseline.2023-12-18.filelist.txt">oa_comm_xml.PMC000xxxxxx.baseline.2023-12-18.filelist.txt</a>,
 <a href="oa_comm_xml.PMC000xxxxxx.baseline.2023-12-18.tar.gz">oa_comm_xml.PMC000xxxxxx.baseline.2023-12-18.tar.gz</a>,
 <a href="oa_comm_xml.PMC001xxxxxx.baseline.2023-12-18.filelist.csv">oa_comm_xml.PMC001xxxxxx.baseline.2023-12-18.filelist.csv</a>,
 <a href="oa_comm_xml.PMC001xxxxxx.baseline.2023-12-18.filelist.txt">oa_comm_xml.PMC001xxxxxx.baseline.2023-12-18.filelist.txt</a>,
 <a href="oa_comm_xml.PMC001xxxxxx.baseline.2023-12-18.tar.gz">oa_comm_xml.PMC001xxxxxx.baseline.2023-12-18.tar.gz</a>]

When extracting elements with an html tag of 'a', we yield only the filenames under this directory (other metadata/formatting removed). As you can see, for each folder of xmls (PMC000, PMC001), there is a filelist in csv and txt format, which contains file metadata. The article XMLs are located inside of the .tar.gz files (compressed folders). I will select only the .tar.gz filenames prior to iteration.

In [4]:
tar_gz_files = [x.text for x in files if str.endswith(x.text, 'tar.gz')] ## .text to remove html tags
tar_gz_files[0:5]

['oa_comm_xml.PMC000xxxxxx.baseline.2023-12-18.tar.gz',
 'oa_comm_xml.PMC001xxxxxx.baseline.2023-12-18.tar.gz',
 'oa_comm_xml.PMC002xxxxxx.baseline.2023-12-18.tar.gz',
 'oa_comm_xml.PMC003xxxxxx.baseline.2023-12-18.tar.gz',
 'oa_comm_xml.PMC004xxxxxx.baseline.2023-12-18.tar.gz']

First, I will show the file retrieval process of just one XML from the first .tar.gz file.

In [5]:
for l in tar_gz_files[0:1]: ## first .gz file
    tar_gz = requests.get(url + l) ## get .tar.gz file from site (url specified above)
    with tarfile.open(fileobj=BytesIO(tar_gz.content)) as xmls: ## decompress tar.gz file, extract metadata
        for xml in xmls.getnames()[0:1]: ## first xml for demonstration here
            xmls.extract(xml) ## locally save single xml
            with open(xml, "rb") as f: ## read contents of locally saved xml
                Bs_data = BeautifulSoup(f, "xml")
str(Bs_data)[0:2000] ## show head of xml

'<?xml version="1.0" encoding="utf-8"?>\n<!DOCTYPE article PUBLIC "-//NLM//DTD JATS (Z39.96) Journal Archiving and Interchange DTD v1.0 20120330//EN" "JATS-archivearticle1.dtd">\n<article article-type="research-article" xmlns:mml="http://www.w3.org/1998/Math/MathML" xmlns:xlink="http://www.w3.org/1999/xlink"><?properties open_access?><front><journal-meta><journal-id journal-id-type="nlm-ta">PLoS Biol</journal-id><journal-id journal-id-type="iso-abbrev">PLoS Biol</journal-id><journal-id journal-id-type="publisher-id">pbio</journal-id><journal-id journal-id-type="pmc">plosbiol</journal-id><journal-title-group><journal-title>PLoS Biology</journal-title></journal-title-group><issn pub-type="ppub">1544-9173</issn><issn pub-type="epub">1545-7885</issn><publisher><publisher-name>Public Library of Science</publisher-name><publisher-loc>San Francisco, USA</publisher-loc></publisher></journal-meta><article-meta><article-id pub-id-type="pmid">12929205</article-id><article-id pub-id-type="pmc">PMC

As you can see, these XMLs have tags as well. Lets check how these tags are distributed across articles so we can identify the metadata we can extract. I will iterate across the first 100 xmls and yield tag counts for each. The XMLs will be extracted using the process shown above, but I will add one tweak to the process: I will delete the locally saved XML after the metadata is parsed in the iterations. This will limit the local storage used during this process.

In [6]:
xml_tag_distribs = [] ## empty list to store parsed xml data for all articles
for l in tar_gz_files[0:1]: ## first .gz file
    tar_gz = requests.get(url + l) ## get .tar.gz file from site (url specified above)
    with tarfile.open(fileobj=BytesIO(tar_gz.content)) as xmls: ## decompress tar.gz file, extract metadata
        for xml in xmls.getnames()[0:100]: ## first 100 xmls
            xmls.extract(xml) ## locally save single xml
            with open(xml, "rb") as f: ## read contents of locally saved xml
                Bs_data = BeautifulSoup(f, "xml") ## save xml contents into beautifulsoup
                tag_counts = pd.Series([tag.name for tag in Bs_data.find_all()]).value_counts() ## get counts of tags
                tag_counts = tag_counts.reset_index().rename(columns = {0:xml+'_count', 'index':'tag'}) ## format tag counts into pd df
                xml_tag_distribs.append(tag_counts)
            os.remove(xml) ## remove file from local instance
            os.rmdir(str.split(xml, '/')[0]) ## remove created folder from local instance

In [7]:
article_tag_counts = reduce(lambda  left,right: pd.merge(left,right,on=['tag'],
                                            how='outer'), xml_tag_distribs)
article_tag_counts.head(3)

,tag,PMC000xxxxxx/PMC176545.xml_count,PMC000xxxxxx/PMC176546.xml_count,PMC000xxxxxx/PMC176547.xml_count,PMC000xxxxxx/PMC176548.xml_count,PMC000xxxxxx/PMC193604.xml_count,PMC000xxxxxx/PMC193605.xml_count,PMC000xxxxxx/PMC193606.xml_count,PMC000xxxxxx/PMC193607.xml_count,PMC000xxxxxx/PMC212319.xml_count,...,PMC000xxxxxx/PMC340961.xml_count,PMC000xxxxxx/PMC340962.xml_count,PMC000xxxxxx/PMC340963.xml_count,PMC000xxxxxx/PMC340964.xml_count,PMC000xxxxxx/PMC350664.xml_count,PMC000xxxxxx/PMC350667.xml_count,PMC000xxxxxx/PMC350672.xml_count,PMC000xxxxxx/PMC350674.xml_count,PMC000xxxxxx/PMC359389.xml_count,PMC000xxxxxx/PMC368155.xml_count
0,given-names,269.0,96.0,NaN,NaN,160.0,209.0,NaN,NaN,215.0,...,NaN,NaN,1.0,NaN,320.0,303.0,NaN,NaN,2.0,19.0
1,surname,269.0,96.0,NaN,NaN,160.0,209.0,NaN,NaN,215.0,...,NaN,NaN,1.0,NaN,320.0,303.0,NaN,NaN,2.0,19.0
2,name,269.0,96.0,NaN,NaN,160.0,209.0,NaN,NaN,215.0,...,NaN,NaN,1.0,NaN,320.0,303.0,NaN,NaN,2.0,19.0


We have extracted a dataframe with rows unique by XML tag, and tag counts for each XML. I will check for tags which are present in all 100 XMLs.

In [8]:
article_tag_counts.dropna()['tag'].tolist()

['p',
 'year',
 'article-title',
 'volume',
 'subject',
 'month',
 'day',
 'journal-id',
 'pub-date',
 'article-id',
 'subj-group',
 'issn',
 'publisher-name',
 'publisher-loc',
 'article',
 'body',
 'license-p',
 'journal-meta',
 'journal-title-group',
 'journal-title',
 'publisher',
 'article-meta',
 'article-categories',
 'title-group',
 'issue',
 'elocation-id',
 'front',
 'permissions',
 'copyright-statement',
 'license']

These are attributes which we can extract for all 100 XMLs. There can be value in looking for more sparse tags if a specific attribute is desired. For this demo, I'll keep it simple. Let's extract article title, journal title, article id, and publication date. I'll first view all occurences of these tags in the last article we iterated over so we can see if there are additional subtypes of these tags.

In [9]:
Bs_data.find_all('article-title')

[<article-title>Identifying Protein Function—A Call for Community Action</article-title>,
 <article-title>Life with 6000 genes</article-title>,
 <article-title>The <italic>hemK</italic> gene in <named-content content-type="genus-species">Escherichia coli</named-content> encodes the N5-glutamine methyltransferase that modifies peptide release factors</article-title>,
 <article-title>HemK, a class of protein methyl transferase with similarity to DNA methyl transferases, methylates polypeptide chain release factors, and <italic>hemK</italic> knockout induces defects in translational termination</article-title>,
 <article-title>Cloning and sequencing of a previously unidentified gene that is involved in the biosynthesis of heme in <named-content content-type="genus-species">Escherichia coli</named-content>
 </article-title>]

In [10]:
Bs_data.find_all('journal-title')

[<journal-title>PLoS Biology</journal-title>]

In [11]:
Bs_data.find_all('article-id')

[<article-id pub-id-type="pmid">15024411</article-id>,
 <article-id pub-id-type="pmc">PMC368155</article-id>,
 <article-id pub-id-type="doi">10.1371/journal.pbio.0020042</article-id>]

In [12]:
Bs_data.find_all('pub-date')

[<pub-date pub-type="ppub"><month>3</month><year>2004</year></pub-date>,
 <pub-date pub-type="epub"><day>16</day><month>3</month><year>2004</year></pub-date>,
 <pub-date pub-type="pmc-release"><day>16</day><month>3</month><year>2004</year></pub-date>]

We see that article-id and pub-date tags have subtypes. In parsing, I will ensure that we are extracting PMC for article-id, and epub (electronic publication date) for pub-date. Now, I'll create a function to exract the desired metadata from an XML read into BeautifulSoup.

In [13]:
def parse_xml(Bs_data):
    ## get article title
    if Bs_data.find('article-title'): ## these if statements are there because .getText raises an error if tag isn't found
        article_title = Bs_data.find('article-title').getText()
    else:
        article_title = None
    ## get journal title
    if Bs_data.find('journal-title'):
        journal_title = Bs_data.find('journal-title').getText()
    else:
        journal_title = None
    ## get pmc
    if Bs_data.find('article-id', {'pub-id-type' : "pmc"}):
        pmc = Bs_data.find('article-id', {'pub-id-type' : "pmc"}).getText()
    else:
        pmc = None
    ## get epub date
    epub_date = Bs_data.find('pub-date', {'pub-type' : 'epub'})
    if (epub_date is not None and epub_date.find('day') and epub_date.find('month') and epub_date.find('year')):
        epub_day = epub_date.find('day').getText()
        epub_month = epub_date.find('month').getText()
        epub_year = epub_date.find('year').getText()
        epub_date = epub_month + '/' + epub_day + '/' + epub_year ## concat date to yield common format
    else:
        epub_date = None
    xml_metadata = {'pmc':pmc, 'article_title':article_title, 'journal_title':journal_title, 'epub_date':epub_date}
    return xml_metadata

In [14]:
parse_xml(Bs_data)

{'pmc': 'PMC368155',
 'article_title': 'Identifying Protein Function—A Call for Community Action',
 'journal_title': 'PLoS Biology',
 'epub_date': '3/16/2004'}

The function successfully extracted PMC, article and journal title, and electronic publication date from the XML data for the article. Next, I will iterate over the first 100 xmls in the first .tar.gz file and extract this metadata.

In [15]:
article_data = [] ## empty list to store parsed xml data for all articles
for l in tar_gz_files[0:1]: ## first .gz file
    tar_gz = requests.get(url + l) ## get .tar.gz file from site (url specified above)
    with tarfile.open(fileobj=BytesIO(tar_gz.content)) as xmls: ## decompress tar.gz file, extract metadata
        for xml in xmls.getnames()[0:100]: ## first 100 xmls
            xmls.extract(xml) ## locally save single xml
            with open(xml, "rb") as f: ## read contents of locally saved xml
                Bs_data = BeautifulSoup(f, "xml") ## save xml contents into beautifulsoup
                parsed_article_data = parse_xml(Bs_data) ## parse xml article metadata
                article_data.append(parsed_article_data)
            os.remove(xml) ## remove file from local instance
            os.rmdir(str.split(xml, '/')[0]) ## remove created folder from local instance

In [16]:
article_metadata_df = pd.DataFrame(article_data)
article_metadata_df.head()

,pmc,article_title,journal_title,epub_date
0,PMC176545,The Transcriptome of the Intraerythrocytic Dev...,PLoS Biology,8/18/2003
1,PMC176546,DNA Analysis Indicates That Asian Elephants Ar...,PLoS Biology,8/18/2003
2,PMC176547,Borneo Elephants: A High Priority for Conserva...,PLoS Biology,8/18/2003
3,PMC176548,Monitoring Malaria: Genomic Activity of the Pa...,PLoS Biology,8/18/2003
4,PMC193604,\nDrosophila Free-Running Rhythms Require Inte...,PLoS Biology,9/15/2003


In [17]:
article_metadata_df.isna().sum()

pmc              0
article_title    0
journal_title    0
epub_date        0
dtype: int64

In [18]:
len(article_metadata_df)

100

The desired metadata from the 100 articles were extracted successfully, and finish the process with no local storage being used.

With respect to PubMed data, there is much more metadata which could be extracted here. There are roughly 36 million XMLs to extract, thus a rich data extraction can be performed using this notebook as a backbone.

This exercise represents a basic ETL process. We extracted XMLs from a public domain, and transformed them into a dataframe containing article metatata. The final step would be loading the data to a storage server. Since this is a demo, we'll just keep this an ET process.